In [73]:
%load_ext autoreload
%autoreload 2
import os, sys
os.environ['STREAM_LOG_LEVEL'] = 'ERROR'
os.environ['FILE_LOG_LEVEL'] = 'DEBUG'
os.environ['PROPAGATE_TO_ROOT_LOGGER'] = 'False'
os.environ['PROPAGATE_TO_ROOT_LOGGER'], os.environ['STREAM_LOG_LEVEL']
from trade.assets.Stock import Stock
from trade.assets.Option import Option
from trade.assets.OptionStructure import OptionStructure
from trade.helpers.Context import Context, clear_context
from trade.helpers.helper import (change_to_last_busday, 
                                  is_USholiday, 
                                  is_busday, 
                                  setup_logger, 
                                  generate_option_tick, 
                                  get_option_specifics_from_key,
                                  identify_length,
                                  extract_numeric_value)
from scipy.stats import percentileofscore
from dbase.DataAPI.ThetaData import (list_contracts, retrieve_openInterest, retrieve_eod_ohlc, retrieve_quote)
from pandas.tseries.offsets import BDay
from itertools import product
import pandas as pd
from copy import deepcopy
import numpy as np
import time
chain_cache = {}
close_cache = {}
spot_cache = {}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
sys.path
from dbase.DataAPI.ThetaData import list_contracts
list_contracts('AAPL', '2025-01-13')

,root,expiration,strike,right
0,AAPL,20250919,215.0,P
1,AAPL,20250919,215.0,C
2,AAPL,20251017,215.0,P
3,AAPL,20250321,350.0,C
4,AAPL,20250124,220.0,P
...,...,...,...,...
1278,AAPL,20250131,217.5,C
1279,AAPL,20250718,215.0,C
1280,AAPL,20250718,215.0,P
1281,AAPL,20250815,215.0,P


In [3]:
class RiskManager:
    def __init__(self,
                 bars,
                 events,
                 initial_capital,
                 ):
        self.bars = bars
        self.events = events
        self.initial_capital = initial_capital
        self.symbol_list = self.bars.symbol_list


    def get_order(self, symbol, date, order_settings):
        pass


In [4]:
def generate_option_order(
        direction,
        dte,
        relative_strike,
):
    pass



In [5]:
generate_option_tick?

Signature: generate_option_tick(symbol, right, exp, strike)
Docstring: <no docstring>
File:      ~/cloned_repos/QuantTools/trade/helpers/helper.py
Type:      function

In [6]:
## Why is AMZN during cheap vol returning two?
def chain_details(date, ticker, tgt_dte, tgt_moneyness, right = 'P', moneyness_width = 0.15, print_stderr = False):
    return_dataframe = pd.DataFrame()
    errors = {}
    if not (is_USholiday(date) and not is_busday(date)):
        try:
            print(date, ticker) if print_stderr else None
            ## Get both calls and puts per moneyness. For 1 Moneyness, both will most be available. If not, if one is False, other True. 
            ## We will need to get two rows. 
            chain_key = f"{date}_{ticker}"
            with Context(end_date = date):
                if chain_key in chain_cache:
                    Option_Chain = chain_cache[chain_key]
                else:
                    start_time = time.time()
                    Stock_obj = Stock(ticker, run_chain = False)
                    end_time = time.time()
                    print(f"Time taken to get stock object: {end_time-start_time}") if print_stderr else None
                    Option_Chain = Stock_obj.option_chain()
                    Spot = Stock_obj.spot(ts = False)
                    Spot = list(Spot.values())[0]
                    Option_Chain['Spot'] = Spot
                    Option_Chain['q'] = Stock_obj.div_yield()
                    Option_Chain['r'] = Stock_obj.rf_rate
                    chain_cache[chain_key] = Option_Chain

                
                Option_Chain_Filtered = Option_Chain[Option_Chain[right.upper()] == True]
                
                
                if right == 'P':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.index.get_level_values('strike')/Option_Chain_Filtered.Spot
                elif right == 'C':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.Spot/Option_Chain_Filtered.index.get_level_values('strike')
                else:
                    raise ValueError(f'Right dne. recieved {right}')
                Option_Chain_Filtered['moneyness_spread'] = (tgt_moneyness-Option_Chain_Filtered['relative_moneyness'])**2
                Option_Chain_Filtered['dte_spread'] = (Option_Chain_Filtered.index.get_level_values('DTE')-tgt_dte)**2
                Option_Chain_Filtered.sort_values(by=['dte_spread','moneyness_spread'], inplace = True)
                Option_Chain_Filtered = Option_Chain_Filtered.loc[Option_Chain_Filtered['dte_spread'] == Option_Chain_Filtered['dte_spread'].min()]
                option_details = Option_Chain_Filtered[(Option_Chain_Filtered['relative_moneyness'] >= tgt_moneyness-moneyness_width) & 
                                                       (Option_Chain_Filtered['relative_moneyness'] <= tgt_moneyness+moneyness_width)]
                option_details['build_date'] = date
                option_details['ticker'] = ticker
                option_details['moneyness'] = tgt_moneyness
                option_details['TGT_DTE'] = tgt_dte
                option_details.reset_index(inplace = True)
                option_details.set_index('build_date', inplace = True)
                option_details['right'] = right
                option_details.drop(columns = ['C','P'], inplace = True)
                option_details['option_id'] = option_details.apply(lambda x: generate_option_tick(symbol = x['ticker'], 
                                                                    exp = x['expiration'].strftime('%Y-%m-%d'), strike = float(x['strike']), right = x['right']), axis = 1)
                return_dataframe = pd.concat([return_dataframe, option_details])
            clear_context()
            return_dataframe.drop_duplicates(inplace = True)

        except Exception as e:
            raise
            errors[date] = e
            return errors
        return return_dataframe.sort_values('relative_moneyness', ascending=False)
    else:
        return None, errors
    

details= chain_details('2024-03-12', 'TSLA', 365, 0.7, moneyness_width = 0.20)

2025-01-19 20:34:19 rates_thread INFO: Waiting for rates data to be ready. 0 seconds elasped
2025-01-19 20:34:24 rates_thread INFO: Waiting for rates data to be ready. 5 seconds elasped


In [7]:
chain_cache.keys()

dict_keys(['2024-03-12_TSLA'])

In [8]:
details

right,expiration,DTE,strike,Spot,q,r,relative_moneyness,moneyness_spread,dte_spread,ticker,moneyness,TGT_DTE,right,option_id
build_date,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,155.0,177.539993,0,0.0524,0.873043,0.029944,64,TSLA,0.7,365,P,TSLA20250321000155P
2024-03-12,2025-03-21,373,150.0,177.539993,0,0.0524,0.844880,0.020990,64,TSLA,0.7,365,P,TSLA20250321000150P
2024-03-12,2025-03-21,373,145.0,177.539993,0,0.0524,0.816717,0.013623,64,TSLA,0.7,365,P,TSLA20250321000145P
2024-03-12,2025-03-21,373,140.0,177.539993,0,0.0524,0.788555,0.007842,64,TSLA,0.7,365,P,TSLA20250321000140P
2024-03-12,2025-03-21,373,135.0,177.539993,0,0.0524,0.760392,0.003647,64,TSLA,0.7,365,P,TSLA20250321000135P
2024-03-12,2025-03-21,373,130.0,177.539993,0,0.0524,0.732229,0.001039,64,TSLA,0.7,365,P,TSLA20250321000130P
2024-03-12,2025-03-21,373,125.0,177.539993,0,0.0524,0.704067,0.000017,64,TSLA,0.7,365,P,TSLA20250321000125P
2024-03-12,2025-03-21,373,120.0,177.539993,0,0.0524,0.675904,0.000581,64,TSLA,0.7,365,P,TSLA20250321000120P
2024-03-12,2025-03-21,373,115.0,177.539993,0,0.0524,0.647741,0.002731,64,TSLA,0.7,365,P,TSLA20250321000115P


In [66]:
order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 30, 'moneyness_width': 0.1},
                {'direction': 'long', 'rel_strike': 1.05, 'dte': 30, 'moneyness_width': 0.1},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 30, 'moneyness_width': 0.1}
            ],
            'name': 'vertical_spread'
        }


tick = 'AAPL'
date = '2024-03-12'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'P'
direction_index = {}

In [69]:
for indx, v in enumerate(order_settings['specifics']):
    direction_index[indx] = v['direction']

direction_index

{0: 'long', 1: 'long', 2: 'short'}

In [82]:
def produce_order_candidates(settings, tick, date, right = 'P'):
    order_candidates = {'long': [], 'short': []}
    for spec in settings['specifics']:
        order_candidates[spec['direction']].append(chain_details(date, tick, spec['dte'], spec['rel_strike'], right,  moneyness_width = spec['moneyness_width']))
        print(spec)
    return order_candidates


def liquidity_check(id, date, pass_threshold = 250):
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():

        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]

    start = (pd.to_datetime(date) - BDay(10)).strftime('%Y-%m-%d')
    oi_data = retrieve_openInterest(**transfer_dict, end_date=date, start_date=start)
    print(f'Open Interest > {pass_threshold} for {id}:', oi_data.Open_interest.mean() )
    return oi_data.Open_interest.mean() > pass_threshold


def available_close_check(id, date, threshold = 0.7):
    cache_key = f"{id}_{date}"
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():
        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]
    
    if cache_key in close_cache:
        close_data_sample = close_cache[cache_key]
    else:
        start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
        close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
        close_cache[cache_key] = close_data_sample
    close_mask_series = close_data_sample.Close != 0
    return close_mask_series.sum()/len(close_mask_series) > threshold


def get_structure_price(tradeables, direction_index, date, tick, right = 'P'):
    structure_price = 0

    for pack in tradeables:

        for id in pack:

            if id not in spot_cache:
                
                cache_key = f"{id}_{date}"
                sample_id = deepcopy(get_option_specifics_from_key(id))
                new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
                transfer_dict = {}
                for k, v in sample_id.items():
                    if k in new_dict_keys:
                        if k == 'strike':
                            transfer_dict[new_dict_keys[k]] = float(sample_id[k])
                        else:
                            transfer_dict[new_dict_keys[k]] = sample_id[k]
                close_data_sample = spot_cache[cache_key]
                start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
                close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
                spot_cache[cache_key] = close_data_sample['Midpoint'][date]

    


In [83]:
get_structure_price([('AAPL20240412000180P', 'AAPL20240412000180P', 'AAPL20240412000160P')], '', date, 'AAPL')

KeyError: 'AAPL20240412000180P_2024-03-12'

In [57]:
order_candidates = produce_order_candidates(order_settings, tick, date, right)

{'direction': 'long', 'rel_strike': 1.0, 'dte': 30, 'moneyness_width': 0.1}
{'direction': 'long', 'rel_strike': 1.05, 'dte': 30, 'moneyness_width': 0.1}
{'direction': 'short', 'rel_strike': 0.85, 'dte': 30, 'moneyness_width': 0.1}


In [53]:
order_candidates['long'][1]

right,expiration,DTE,strike,Spot,q,r,relative_moneyness,moneyness_spread,dte_spread,ticker,moneyness,TGT_DTE,right,option_id
build_date,,,,,,,,,,,,,,
2024-03-12,2024-04-12,30,185.0,173.229996,0.005542,0.0524,1.067944,0.000322,0,AAPL,1.05,30,P,AAPL20240412000185P
2024-03-12,2024-04-12,30,180.0,173.229996,0.005542,0.0524,1.039081,0.000119,0,AAPL,1.05,30,P,AAPL20240412000180P
2024-03-12,2024-04-12,30,175.0,173.229996,0.005542,0.0524,1.010218,0.001583,0,AAPL,1.05,30,P,AAPL20240412000175P
2024-03-12,2024-04-12,30,170.0,173.229996,0.005542,0.0524,0.981354,0.004712,0,AAPL,1.05,30,P,AAPL20240412000170P
2024-03-12,2024-04-12,30,165.0,173.229996,0.005542,0.0524,0.952491,0.009508,0,AAPL,1.05,30,P,AAPL20240412000165P


In [58]:
# for direction in order_candidates:
#     for i,data in enumerate(order_candidates[direction]):
#         data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
#         data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
#         order_candidates[direction][i] = data[data.liquidity_check == True]

# for direction in order_candidates:
#     for i,data in enumerate(order_candidates[direction]):
#         data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
#         order_candidates[direction][i] = data[data.available_close_check == True]  


for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
        data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))


## Filter Unique Combinations per leg.

unique_ids = {'long': [], 'short': []}
for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        unique_ids[direction].append(data.option_id.unique())

tradeable_ids = list(product(*unique_ids['long'], *unique_ids['short']))
  

Open Interest > 250 for AAPL20240412000185P: 196.0
Open Interest > 250 for AAPL20240412000180P: 283.0
Open Interest > 250 for AAPL20240412000175P: 1127.5
Open Interest > 250 for AAPL20240412000170P: 672.75
Open Interest > 250 for AAPL20240412000165P: 587.0
Open Interest > 250 for AAPL20240412000160P: 870.625
Open Interest > 250 for AAPL20240412000185P: 196.0
Open Interest > 250 for AAPL20240412000180P: 283.0
Open Interest > 250 for AAPL20240412000175P: 1127.5
Open Interest > 250 for AAPL20240412000170P: 672.75
Open Interest > 250 for AAPL20240412000165P: 587.0
Open Interest > 250 for AAPL20240412000160P: 870.625
Open Interest > 250 for AAPL20240412000155P: 235.125
Open Interest > 250 for AAPL20240412000150P: 135.25
Open Interest > 250 for AAPL20240412000145P: 205.0
Open Interest > 250 for AAPL20240412000140P: 47.875
Open Interest > 250 for AAPL20240412000135P: 96.375
Open Interest > 250 for AAPL20240412000130P: 12.875


In [59]:
## Keep only both True
unique_ids = {'long': [], 'short': []}
for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        unique_ids[direction].append(data[(data.liquidity_check == True) & (data.available_close_check == True)].option_id.unique().tolist())

tradeable_ids = list(product(*unique_ids['long'], *unique_ids['short']))
tradeable_ids, unique_ids

([('AAPL20240412000180P', 'AAPL20240412000180P', 'AAPL20240412000160P'),
  ('AAPL20240412000180P', 'AAPL20240412000175P', 'AAPL20240412000160P'),
  ('AAPL20240412000180P', 'AAPL20240412000170P', 'AAPL20240412000160P'),
  ('AAPL20240412000180P', 'AAPL20240412000165P', 'AAPL20240412000160P'),
  ('AAPL20240412000175P', 'AAPL20240412000180P', 'AAPL20240412000160P'),
  ('AAPL20240412000175P', 'AAPL20240412000175P', 'AAPL20240412000160P'),
  ('AAPL20240412000175P', 'AAPL20240412000170P', 'AAPL20240412000160P'),
  ('AAPL20240412000175P', 'AAPL20240412000165P', 'AAPL20240412000160P'),
  ('AAPL20240412000170P', 'AAPL20240412000180P', 'AAPL20240412000160P'),
  ('AAPL20240412000170P', 'AAPL20240412000175P', 'AAPL20240412000160P'),
  ('AAPL20240412000170P', 'AAPL20240412000170P', 'AAPL20240412000160P'),
  ('AAPL20240412000170P', 'AAPL20240412000165P', 'AAPL20240412000160P'),
  ('AAPL20240412000165P', 'AAPL20240412000180P', 'AAPL20240412000160P'),
  ('AAPL20240412000165P', 'AAPL20240412000175P', 'A

In [72]:
filtered = [t for t in tradeable_ids if len(set(t)) == len(t)]
pd.DataFrame(filtered)

,0,1,2
0,AAPL20240412000180P,AAPL20240412000175P,AAPL20240412000160P
1,AAPL20240412000180P,AAPL20240412000170P,AAPL20240412000160P
2,AAPL20240412000180P,AAPL20240412000165P,AAPL20240412000160P
3,AAPL20240412000175P,AAPL20240412000180P,AAPL20240412000160P
4,AAPL20240412000175P,AAPL20240412000170P,AAPL20240412000160P
5,AAPL20240412000175P,AAPL20240412000165P,AAPL20240412000160P
6,AAPL20240412000170P,AAPL20240412000180P,AAPL20240412000160P
7,AAPL20240412000170P,AAPL20240412000175P,AAPL20240412000160P
8,AAPL20240412000170P,AAPL20240412000165P,AAPL20240412000160P
9,AAPL20240412000165P,AAPL20240412000180P,AAPL20240412000160P


In [275]:
order_candidates['long'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check,available_close_check
build_date,,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,180.0,1.039081,0.001527,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000180P,True,True
2024-03-12,2025-03-21,373,170.0,0.981354,0.000348,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000170P,True,True
2024-03-12,2025-03-21,373,165.0,0.952491,0.002257,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000165P,True,True


In [277]:
order_candidates['short'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check,available_close_check
build_date,,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,150.0,0.865901,0.000253,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000150P,True,True


In [260]:
structure = {
    'long': [{'strike': 165.0, 'expiration': '2025-03-21', 'underlier': 'AAPL', 'right': 'p' }],
    'short': [{'strike': 150.0, 'expiration': '2025-03-21', 'underlier': 'AAPL', 'right': 'p' }]
}

with Context(end_date = date):
    struct_obj = OptionStructure(structure)

In [262]:
struct_obj.pv

4.199999999999999

In [227]:
transfer_dict

{'symbol': 'AAPL', 'right': 'P', 'exp': '2025-03-21', 'strike': 165.0}

In [74]:
retrieve_quote(**transfer_dict, end_date = date, start_date=date, start_time = '9:30',)

NameError: name 'transfer_dict' is not defined

In [240]:
57600000/3_600_000

16.0

In [249]:
def identify_length(string, integer, rt=False):
    if rt:
        TIMEFRAMES_VALUES = {'m': 1, 'h': 60, 'd': 60*24, 'w': 60*24*7}
    else:
        TIMEFRAMES_VALUES = {'d': 1, 'w': 5, 'm': 30, 'y': 252, 'q': 91}
    assert string in TIMEFRAMES_VALUES.keys(
    ), f'Available timeframes are {TIMEFRAMES_VALUES.keys()}, recieved "{string}"'
    return integer * TIMEFRAMES_VALUES[string]
identify_length(*extract_numeric_value('1h'), rt=True)*60000


def convert_time_to_miliseconds(time):
    time_obj = pd.to_datetime(time)
    hour = time_obj.hour * 3_600_000
    minute = time_obj.minute * 60_000
    secs = time_obj.second * 1_000
    mili = time_obj.microsecond
    return hour + minute + secs + mili

convert_time_to_miliseconds('9:30')

34200000

In [139]:
order_candidates['long'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check
build_date,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,190.0,1.096808,0.009372,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000190P,False
2024-03-12,2025-03-21,373,185.0,1.067944,0.004616,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000185P,False
2024-03-12,2025-03-21,373,180.0,1.039081,0.001527,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000180P,True
2024-03-12,2025-03-21,373,175.0,1.010218,0.000104,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000175P,False
2024-03-12,2025-03-21,373,170.0,0.981354,0.000348,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000170P,True
2024-03-12,2025-03-21,373,165.0,0.952491,0.002257,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000165P,True
2024-03-12,2025-03-21,373,160.0,0.923628,0.005833,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000160P,False


In [137]:
oi = liquidity_check('AAPL20250321000185P', date)
oi

False

In [75]:
sample_id = deepcopy(get_option_specifics_from_key('AAPL20250321000165P'))
new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
transfer_dict = {}
for k, v in sample_id.items():
    print(k)
    if k in new_dict_keys:
        if k == 'strike':
            transfer_dict[new_dict_keys[k]] = float(sample_id[k])
        else:
            transfer_dict[new_dict_keys[k]] = (sample_id[k])

sample_id

ticker
put_call
exp_date
strike


{'ticker': 'AAPL', 'put_call': 'P', 'exp_date': '2025-03-21', 'strike': 165.0}

In [76]:
close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
close_mask_series = close_data_sample.Close != 0
close_mask_series.sum()/len(close_mask_series)
# close_data_sample.Close

0.8

In [210]:
available_close_check('AAPL20250321000160P', date)

0.7333333333333333

In [81]:
close_data_sample['Midpoint'][date]

9.85

In [98]:
sample_id = deepcopy(get_option_specifics_from_key('AAPL20250321000160P'))
sample_id

{'ticker': 'AAPL', 'put_call': 'P', 'exp_date': '2025-03-21', 'strike': 160}

In [ ]:
retrieve_openInterest

Signature:
retrieve_openInterest(
    symbol,
    end_date: str,
    exp: str,
    right: str,
    start_date: str,
    strike: float,
    print_url=False,
    proxy='http://18.232.166.224:5500/thetadata',
)
Docstring: Interval size in miliseconds. 1 minute is 6000
File:      ~/cloned_repos/FinanceDatabase/dbase/DataAPI/ThetaData.py
Type:      function

In [37]:

chain_details?

Signature:
chain_details(
    date,
    ticker,
    tgt_dte,
    tgt_moneyness,
    right='P',
    moneyness_width=0.15,
)
Docstring: <no docstring>
File:      /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_16186/3700718713.py
Type:      function

Steps to producing an order:

- S1: RM recieves order settings from PM
- S2: RM produces a dataframe of potential options based on settings (if two legs produce two dataframes)
- S3: RM assesses if option passes all checks
    - C1: Minimum Available close
    - C2: Liquidity (Open Interest)
    - C2.5: (for Spreads only) Ensure both legs are not the same
    - Optional, to extend:
    - C3: Bid-Ask Spread
    
- S4: Return picked order to portfolio manager, which places the order. 
- Example:
    {'long': [optionid or {'strike', 'exp'}], 'short' : []}

In [65]:
generate_option_tick('AAPL','P' ,'2025-03-21', 150.0)
# save_option_keys(generate_option_tick('AAPL','P' ,'2025-03-21', 150.0), {'ticker':'AAPL', 'put_call':'P', 'exp_date':'2025-03-21', 'strike':150.0})

'AAPL20250321000150P'

In [256]:
opt = Option(**option_keys['AAPL20250321000150P'], run_chain = True)

NameError: name 'option_keys' is not defined

In [39]:
import_option_keys()
option_keys

{}